<a href="https://colab.research.google.com/github/keripikkaneboo/Hands-On-Machine-Learning-O-Reilly-/blob/main/14.%20Chapter14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bab 14: Deep Computer Vision with Convolutional Neural Networks (CNN)

Bab ini menyelami dunia **Computer Vision** dengan menggunakan **Convolutional Neural Networks (CNNs)**, arsitektur yang menjadi dasar dari sebagian besar kemajuan di bidang ini.

* **Inspirasi dari Korteks Visual**: Arsitektur CNN terinspirasi dari cara kerja korteks visual otak manusia. Neuron di korteks visual memiliki *local receptive field*, yang berarti mereka hanya merespons stimulus di area visual yang terbatas. Lapisan-lapisan neuron ini tersusun secara hierarkis, di mana lapisan yang lebih tinggi belajar mengenali pola yang lebih kompleks dengan menggabungkan pola dari lapisan di bawahnya.

* **Blok Pembangun CNN**:
    * **Convolutional Layer**: Lapisan ini adalah inti dari CNN. Alih-alih terhubung ke setiap piksel input, neuron di lapisan ini hanya terhubung ke piksel dalam *receptive field* lokal mereka.
        * ***Filters*** (atau *kernels*): Sebuah lapisan konvolusional menerapkan beberapa *filter* ke inputnya. Setiap *filter* bertindak sebagai pendeteksi fitur (misalnya, mendeteksi tepi vertikal, tekstur, atau bentuk tertentu).
        * ***Feature Maps***: Output dari satu *filter* yang diterapkan pada seluruh gambar disebut *feature map*.
        * Bobot (*weights*) dalam satu *filter* digunakan bersama (*shared*) di seluruh gambar, yang secara drastis mengurangi jumlah parameter.
    * **Pooling Layer**: Tujuannya adalah untuk melakukan *subsampling* (mengecilkan) *feature map* untuk mengurangi beban komputasi, penggunaan memori, dan jumlah parameter, serta memberikan sedikit invarian terhadap translasi kecil.
        * ***Max Pooling***: Jenis yang paling umum, mengambil nilai maksimum dari setiap *receptive field*.
        * ***Average Pooling***: Mengambil nilai rata-rata.

* **Arsitektur CNN Klasik**: Arsitektur CNN yang umum biasanya terdiri dari tumpukan beberapa lapisan konvolusional (diikuti fungsi aktivasi ReLU) dan lapisan *pooling*. Seiring data mengalir lebih dalam, gambar menjadi lebih kecil secara spasial (tinggi dan lebar) tetapi lebih dalam (memiliki lebih banyak *feature map*). Di bagian akhir, terdapat beberapa lapisan *fully-connected* (*Dense*) untuk melakukan klasifikasi akhir.

* **Arsitektur CNN Terkenal**: Bab ini membahas evolusi arsitektur CNN yang berpengaruh:
    * **LeNet-5**: Pelopor awal untuk pengenalan digit.
    * **AlexNet**: Pemenang kompetisi ImageNet 2012, yang mempopulerkan CNN.
    * **GoogLeNet (Inception)**: Memperkenalkan *inception module* yang memungkinkan jaringan menjadi lebih dalam dengan parameter yang lebih sedikit.
    * **ResNet (Residual Network)**: Memperkenalkan *skip connections* (koneksi residual) yang memungkinkan training jaringan yang sangat dalam (lebih dari 100 lapisan) dengan mengatasi masalah *vanishing gradient*.
    * **Xception & SENet**: Arsitektur yang lebih modern dan efisien.

* **Transfer Learning**: Menggunakan kembali lapisan-lapisan bawah dari model yang sudah dilatih pada dataset besar (seperti ImageNet) untuk tugas baru dengan data yang lebih sedikit. Ini adalah teknik yang sangat efektif dan umum digunakan.

* **Tugas Computer Vision Lanjutan**:
    * **Object Detection**: Mengklasifikasikan dan melokalisasi beberapa objek dalam gambar dengan memberikan *bounding box* di sekitar setiap objek.
    * **Semantic Segmentation**: Mengklasifikasikan setiap piksel dalam gambar.

### 1. Membangun CNN Sederhana untuk Klasifikasi
Kita akan membangun CNN untuk mengklasifikasikan dataset Fashion MNIST.

```python
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Memuat dan mempersiapkan data Fashion MNIST
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

# Normalisasi piksel dan menambahkan dimensi channel
X_train = np.expand_dims(X_train, axis=-1).astype(np.float32) / 255
X_valid = np.expand_dims(X_valid, axis=-1).astype(np.float32) / 255
X_test = np.expand_dims(X_test, axis=-1).astype(np.float32) / 255

# Membangun model CNN
model = keras.models.Sequential([
    # Conv -> Pool -> Conv -> Pool -> Flatten -> Dense -> Output
    keras.layers.Conv2D(64, kernel_size=7, activation="relu", padding="same",
                        input_shape=[28, 28, 1]),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, kernel_size=3, activation="relu", padding="same"),
    keras.layers.Conv2D(128, kernel_size=3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
# history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
# model.evaluate(X_test, y_test)
print("Model CNN sederhana telah dibuat dan dikompilasi.")
```

### 2. Transfer Learning Menggunakan Model Pretrained
Ini adalah cara yang paling umum dan efektif. Kita akan menggunakan model Xception yang sudah dilatih pada ImageNet untuk mengklasifikasikan bunga.

```python
# Diperlukan untuk Google Colab
!pip install -q -U tensorflow_datasets

import tensorflow_datasets as tfds

# Memuat dataset bunga dan informasinya
dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)
dataset_size = info.splits["train"].num_examples
n_classes = info.features["label"].num_classes

# Membagi dataset
test_split, valid_split, train_split = tfds.Split.TRAIN.subsplit([10, 15, 75])
test_set = tfds.load("tf_flowers", split=test_split, as_supervised=True)
valid_set = tfds.load("tf_flowers", split=valid_split, as_supervised=True)
train_set = tfds.load("tf_flowers", split=train_split, as_supervised=True)

# Fungsi preprocessing
def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

# Membangun pipeline data
batch_size = 32
train_set = train_set.shuffle(1000).map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set.map(preprocess).batch(batch_size).prefetch(1)

# Memuat model dasar Xception tanpa lapisan atas
base_model = keras.applications.xception.Xception(weights="imagenet", include_top=False)

# Menambahkan lapisan atas kustom
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

# Membekukan lapisan dasar agar tidak ikut terlatih pada awalnya
for layer in base_model.layers:
    layer.trainable = False

# Kompilasi dan training
optimizer = keras.optimizers.SGD(learning_rate=0.2, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

# history = model.fit(train_set, validation_data=valid_set, epochs=5)
print("\nModel Transfer Learning telah dibuat dan dikompilasi.")
print("Untuk menjalankannya, hapus komentar pada baris `history = model.fit(...)`.")
```

### 3. Klasifikasi dan Lokalisasi
Model ini tidak hanya memprediksi kelas objek, tetapi juga *bounding box* di sekitarnya. Ini dilakukan dengan memiliki dua *output head*.

```python
# Menggunakan kembali base_model dari contoh sebelumnya
# Misalkan Xception sudah dimuat

# Menambahkan DUA output head
# Head pertama untuk klasifikasi kelas
class_output = keras.layers.Dense(n_classes, activation="softmax")(avg)
# Head kedua untuk regresi bounding box (4 nilai: x, y, tinggi, lebar)
loc_output = keras.layers.Dense(4)(avg)

# Membuat model dengan dua output
model_loc = keras.Model(inputs=base_model.input,
                        outputs=[class_output, loc_output])

# Mengompilasi dengan dua loss function dan bobot loss
model_loc.compile(loss=["sparse_categorical_crossentropy", "mse"],
                  loss_weights=[0.8, 0.2], # Memberi bobot lebih pada loss klasifikasi
                  optimizer=optimizer, metrics=["accuracy"])

# Untuk melatih model ini, targetnya juga harus berupa tuple: (y_class, y_bbox)
# history = model_loc.fit(train_set_with_bbox, validation_data=valid_set_with_bbox, epochs=10)
print("\nModel Klasifikasi & Lokalisasi telah dibuat.")
```

